In [2]:
!pip install standard_transform

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for standard_transform: filename=standard_transform-1.4.0-py3-none-any.whl size=22356 sha256=14a55a08c78645a1267057764809295f03300893dda14c0cd85269ea62081c21
  Stored in directory: /home/vscode/.cache/pip/wheels/ad/3f/3f/2d6f4450cc3c31eb60e2dc6c4a24648a2e9563b610b26c9b67
Successfully built standard_transform

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import gc
import shutil
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
#from sklearn.metrics import r2_score
from ccmodels.preprocessing.connectomics import client_version, subset_v1l234
from ccmodels.preprocessing.selectivity import orientation_extractor,orientation_extractor_v2, orientation_extractor_v3, von_mises, von_mises_single, is_selective, cell_area_identifiers, fpd_assignment, identify_multiscan
from microns_phase3 import nda


In [3]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

#Identify neurons that have been scaned multiple vs one time only
multiscan_ids = identify_multiscan(v1l234_neur)
v1l234_singlescan = v1l234_neur[~v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_multiscan = v1l234_neur[v1l234_neur['pt_root_id'].isin(multiscan_ids)]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [4]:
v1l234_singlescan[(v1l234_singlescan['session'] == 6)&(v1l234_singlescan['scan_idx'] == 2)]

,pt_root_id,id,session,scan_idx,unit_id,pt_position,brain_area,pial_distances,cortex_layer
6,864691135065322052,10219,6,2,2154,"[233744, 113648, 20003]",V1,"[891.7978306358846, 137.67963571587282, 800.12...",L2/3
31,864691135494448016,9690,6,2,2262,"[243232, 104192, 26468]",V1,"[932.9019906315499, 103.30730220400967, 1058.7...",L2/3
35,864691135100947488,120,6,2,4444,"[208096, 148896, 18072]",V1,"[777.3079616877784, 269.1936366292564, 722.880...",L2/3
36,864691135919483824,12251,6,2,4830,"[185568, 148912, 21272]",V1,"[687.5332870857992, 261.4036147994572, 850.880...",L2/3
41,864691135577818373,6601,6,2,5335,"[131456, 152960, 16001]",V1,"[470.49771128666686, 258.6693131447137, 640.04...",L2/3
...,...,...,...,...,...,...,...,...,...
9112,864691135490541159,9900,6,2,4891,"[205488, 145936, 22472]",V1,"[767.9475825914176, 256.4894826955066, 898.880...",L2/3
9116,864691136903807154,12270,6,2,1607,"[227328, 106896, 23959]",V1,"[868.5853922041867, 108.53764432793496, 958.36...",L2/3
9126,864691135816588623,7953,6,2,4404,"[243552, 145536, 27135]",V1,"[919.7636517324706, 268.16555594634724, 1085.4...",L2/3
9149,864691135865514460,645,6,2,2216,"[179568, 110976, 23885]",V1,"[676.8499753590978, 108.14530870627968, 955.40...",L2/3


In [6]:
session_scan_pairs = [(8,7), (4, 7), (5, 3), (5, 6),(5, 7),(6, 2),(6, 4),(6, 6),(6, 7),(7, 3),(7, 4),(7, 5),(8, 5), (9, 3), (9, 4), (9, 6)]

for sess, scan in session_scan_pairs:
    sub = v1l234_singlescan[(v1l234_singlescan['session'] == sess)&(v1l234_singlescan['scan_idx'] == scan)]
    print(f'{sess}, '+f'{scan}')
    print(sub.shape[0])
    print('------------')

8, 7
2
------------
4, 7
378
------------
5, 3
69
------------
5, 6
279
------------
5, 7
340
------------
6, 2
409
------------
6, 4
431
------------
6, 6
315
------------
6, 7
408
------------
7, 3
231
------------
7, 4
412
------------
7, 5
164
------------
8, 5
356
------------
9, 3
624
------------
9, 4
469
------------
9, 6
180
------------


In [14]:
fpd = fpd_assignment(6,6)

In [15]:
unit_key = {'session':6, 'scan_idx':6, 'unit_id':4259}


In [16]:
dft = orientation_extractor_v3(unit_key, fpd)


In [17]:
dft

,orientation,mean_activity,trial_id,monet2_trial_id,condition_hash,radians,session,scan_idx,unit_id
0,202.5,1.455661e-07,0,98,RXu0Jp82MXNjAC70IQuO,3.534292,6,6,4259
1,67.5,4.213260e-01,0,98,RXu0Jp82MXNjAC70IQuO,1.178097,6,6,4259
2,180.0,1.414015e+01,0,98,RXu0Jp82MXNjAC70IQuO,3.141593,6,6,4259
3,22.5,5.207281e+01,0,98,RXu0Jp82MXNjAC70IQuO,0.392699,6,6,4259
4,0.0,3.430655e+01,0,98,RXu0Jp82MXNjAC70IQuO,0.000000,6,6,4259
...,...,...,...,...,...,...,...,...,...
635,45.0,2.326783e-07,39,463,SzeVJsDzE+Ceeb3XK4TT,0.785398,6,6,4259
636,157.5,3.151316e-07,39,463,SzeVJsDzE+Ceeb3XK4TT,2.748894,6,6,4259
637,292.5,1.257619e-06,39,463,SzeVJsDzE+Ceeb3XK4TT,5.105088,6,6,4259
638,22.5,6.114198e-08,39,463,SzeVJsDzE+Ceeb3XK4TT,0.392699,6,6,4259


In [3]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [4]:
multiscan_ids = identify_multiscan(v1l234_neur)

In [5]:
v1l234_singlescan = v1l234_neur[~v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_multiscan = v1l234_neur[v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_singlescan.shape, v1l234_multiscan.shape, v1l234_neur.shape

((5067, 9), (2025, 9), (7092, 9))

In [6]:
#Container with saved data
data = []
            
#Von Mises
columns = ['root_id', 'session', 'scan_idx','cell_id', 'pvalue','tuning_type', 
        'r_squared_diff', 'mean_r_sqrd', 'A', 'phi', 'k', 'activity', 'orientations']

for mult_id in tqdm(multiscan_ids, desc = f'Extracting multiscan neurons data') :
    subset_multiscan = v1l234_multiscan[v1l234_multiscan['pt_root_id'] == mult_id]
    scan_acts = []
    
    for row in range(subset_multiscan.shape[0]):
        unit_key = {'session':subset_multiscan['session'].values[row], 'scan_idx':subset_multiscan['scan_idx'].values[row], 'unit_id':subset_multiscan['unit_id'].values[row]}
        fpd = fpd_assignment(subset_multiscan['session'].values[row], subset_multiscan['scan_idx'].values[row])
        df = orientation_extractor(unit_key, fpd)
        scan_acts.append(df)
    
    all_acts = pd.concat(scan_acts, axis = 1)

    #Calculating max activity for constraining von mises, for plotting data and fit
    gp = df.groupby('radians').mean().reset_index()
    max_act_rad = gp.sort_values('mean_activity', ascending = False)['radians'].values[0]
            
    #Using Von Mises
    pars_d, pcov_d = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')

    pars_s, pcov_s = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')



    # Von Mises activities
    ate_d= [von_mises(i,pars_d[0], max_act_rad,  pars_d[1]) for i in gp['radians']]
    ate_s= [von_mises_single(i,pars_s[0], max_act_rad,  pars_s[1]) for i in gp['radians']]
            
    r2d = r2_score(gp['mean_activity'], ate_d)
    r2s = r2_score(gp['mean_activity'], ate_s)

    #Calculate difference in r2 score between the two model fist to identify fringe cases
    rdiff = r2d-r2s
            
    #single
    vs, ps, max_rad = is_selective(df, max_act_rad)


    #double
    vdb, pdb, max_rad = is_selective(df, max_act_rad, single = False)
            
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],ps,'single', rdiff, r2s, 
                        pars_d[0], max_act_rad,  pars_d[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],pdb,'double', rdiff,r2d, 
                        pars_s[0], max_act_rad,  pars_s[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
#Save the data
data_df = pd.DataFrame(data, columns = columns)
data_df.to_pickle(f'../data/in_processing/orientation_fits/multiscan_fits.pkl')


Extracting multiscan neurons data:   0%|          | 0/967 [00:00<?, ?it/s]

OSError: Cannot save file into a non-existent directory: '../data/in_processing/orientation_fits'

In [18]:
t = pd.read_pickle('/workspaces/microns_phase3_nda/data/in_processing/orientation_fits.pkl')

In [11]:
nda.Scan().fetch(format = 'frame')

nframes  nfields      fps
session scan_idx                           
4       7           40000        8  6.30090
        9           35112        8  6.30090
        10          40000        8  6.30090
5       3           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
6       2           40000        8  6.30090
        4           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
7       3           40000        8  6.30090
        4           40000        8  6.30090
        5           40000        8  6.30090
8       5           40000        8  6.30090
        7           40000        8  6.30090
        9           40000        8  6.30090
9       3           50000        6  8.61754
        4           50000        6  8.61754
        6           57000        4  9.62246

In [15]:
ors_all = pd.read_pickle(f"../data/in_processing/orientation_fits/{os.listdir('../data/in_processing/orientation_fits')[0]}")

#Loading the rest iteratively and concatenating
for file in tqdm(os.listdir('../data/in_processing/orientation_fits/')[1:], desc='Aggregating session, scan files'):
    if file!='.DS_Store':
        cont_df = pd.read_pickle(f'../data/in_processing/orientation_fits/{file}')
        ors_all = pd.concat([ors_all, cont_df], axis = 0)
    else:
        continue

#Saving the data
ors_all.to_pickle('../data/in_processing/orientation_fits.pkl')

Aggregating session, scan files:   0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
shutil.rmtree('../data/in_processing/orientation_fits')